<center><h2>ALTEGRAD Project</h2>

<hr>
<span style="font-variant: small-caps;">Xavier Jiménez</span><br>
<hr>
</center>

# Imports

In [1]:
import networkx as nx
import os
import csv
import numpy as np
import pandas as pd
from random import randint
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

# !pip install pip install karateclub
from gensim.models.doc2vec import Doc2Vec
from os import path
import pickle
from scipy import spatial
import random


In [2]:
if not path.isdir('data'):
    !mkdir data
    !wget -O altegrad.zip https://www.dropbox.com/sh/fhfjjtk0sr7pmse/AAD4ZEtHv9OI5HfVO22tdMX0a?dl=1
    !unzip altegrad.zip
else:
    print('Data already downloaded')

Data already downloaded


# Preprocessing

In [2]:
# Create a graph
G = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)

Number of nodes: 138499
Number of edges: 1091955


In [3]:
# Read the abstract of each paper
try:
    a_file = open("data/authors_preprocessed.pkl", "rb")
    authors = pickle.load(a_file)
    a_file.close()
    print('Authors already preprocessed')
except:
    print('Preprocessing authors')
    authors = dict()
    with open('data/authors.txt', 'r', encoding="utf-8") as f:
        for line in tqdm(f):
            node, author = line.split('|--|')
            # author = author.lower()
            author = author.split(',')
            author[-1] = author[-1].strip()
            authors[int(node)] = author
        a_file = open("data/authors_preprocessed.pkl", "wb")
        pickle.dump(authors, a_file)
        a_file.close()
    print('Preprocessing Done')

authors[0]

Authors already preprocessed


['James H. Niblock', 'Jian-Xun Peng', 'Karen R. McMenemy', 'George W. Irwin']

In [4]:
# Read the abstract of each paper
try:
    a_file = open("data/abstract_preprocessed.pkl", "rb")
    abstracts = pickle.load(a_file)
    a_file.close()
    print('Abstract already preprocessed')
except:
    print('Preprocessing abstracts')
    stop_words = stopwords.words('english')
    porter = PorterStemmer()
    abstracts = dict()
    with open('data/abstracts.txt', 'r', encoding="utf-8") as f:
        for line in tqdm(f):
            node, abstract = line.split('|--|')
            abstract = abstract.lower()
            abstract = "".join([char for char in abstract if char not in string.punctuation])
            abstract = word_tokenize(abstract)
            abstract = [word for word in abstract if word not in stop_words]
            # abstract = [porter.stem(word) for word in abstract]
            abstracts[int(node)] = abstract
    a_file = open("data/abstract_preprocessed.pkl", "wb")
    pickle.dump(abstracts, a_file)
    a_file.close()

print('Preprocessing Done')
abstracts[0][:10]

Abstract already preprocessed
Preprocessing Done


['development',
 'automated',
 'system',
 'quality',
 'assessment',
 'aerodrome',
 'ground',
 'lighting',
 'agl',
 'accordance']

# Training

In [5]:
def remove_random_edges_from_G(p):
    """Removes p lines from edgelist randomly and saves
    remaining lines as edgelist_missing.txt

    Args:
        p (float): line percentage to be removed
    """

    H = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
    with open('data/edgelist.txt') as file:
        lines = file.readlines()

    random.seed(42)
    indices_to_delete = random.sample(range(len(lines)), int(p * len(lines)))

    # sort to delete biggest index first 
    indices_to_delete.sort(reverse=True)

    for i in tqdm(indices_to_delete):
        line = lines[i]
        t = line.split(',')
        H.remove_edge(int(t[0]), int(t[1]))
        
    return H

In [10]:
def validation_score(model, X_train, y_train, cv, scoring = 'log_loss', n_jobs = None, verbose = 0):
    """Computes scores using cross validation for a given model.

    Args:
        model: classifier
        X_train (array like): training set.
        y_train (array like): training lavels.
        cv (int): number of splits.
        scoring (str, optional): Metric. Defaults to 'neg_log_loss'.
        n_jobs (int, optional): Number of cores. Defaults to None.
        verbose (int, optional): Verbose level for cross_val_score

    Returns:
        list: list containing scores from cross validation
    """
    print('Cross validation')
    if scoring == 'log_loss':
        scoring = make_scorer(log_loss, greater_is_better = False, needs_proba = True)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = scoring, n_jobs = n_jobs, verbose = verbose)
    print('Score: {:.2f} ± {:.2f}'.format(-scores.mean(), scores.std()/np.sqrt(cv)))
    
    return scores

def create_submission(model, X_test, X_train, y_train):
    # # Read test data. Each sample is a pair of nodes
    print('Creating submission')
    node_pairs = list()
    with open('data/test.txt', 'r') as f:
        for line in f:
            t = line.split(',')
            node_pairs.append((int(t[0]), int(t[1])))

    
    # Use model to predict if two nodes are linked by an edge
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    y_pred = y_pred[:,1]

    # Write predictions to a file
    predictions = zip(range(len(y_pred)), y_pred)
    os.remove("data/submission.csv")
    with open("data/submission.csv","w") as pred:
        csv_out = csv.writer(pred)
        csv_out.writerow(['id','predicted'])
        for row in predictions:
            csv_out.writerow(row)
    print('Submision created')


def graph_properties(G):
    print('Computing graph properties')
    avg_neighbor_degree = nx.average_neighbor_degree(G)
    pagerank = nx.pagerank_scipy(G)

    return [avg_neighbor_degree, pagerank]
            
def create_dataset(G, H, G_params): 
        
    node_pairs = list()
    with open('data/test.txt', 'r') as f:
        for line in f:
            t = line.split(',')
            node_pairs.append((int(t[0]), int(t[1])))
            
    X_train = np.zeros((2*len(G.edges()), 8 + 2*len(G_params)))
    y_train = np.zeros(2*len(G.edges()))
    X_test = np.zeros((len(node_pairs), 8 + 2*len(G_params)))
    
    n2v_parameters = {'walk_number': 10, 'walk_length': 15, 'dimensions': 64, 'window_size': 5, 'workers': 7}
    d2v_parameters = {'vector_size':128, 'window':5, 'min_count':2, 'epochs':100, 'workers':7}
    
    d2v = Doc2Vec.load("data/abstracts_embedding_doc2vec_vs{:d}_w{:d}_mc{:d}_e{:d}".format(d2v_parameters['vector_size'], d2v_parameters['window'],
                                                                               d2v_parameters['min_count'], d2v_parameters['epochs']))
    n2v = np.load('data/embedding_n2v_wn{:d}_wl{:d}_d{:d}_ws{:d}.npy'.format(n2v_parameters['walk_number'], n2v_parameters['walk_length'],
                                                               n2v_parameters['dimensions'], n2v_parameters['window_size']))
    
    # gae = pd.read_csv('data/Z_GAE_32_tanh_Ep2000_0.1063.csv')
    gae = None
    
    distance = spatial.distance.cosine #euclidean_distance #
    
    for i,edge in tqdm(enumerate(G.edges())):
        X_train, y_train = fill_matrix(X_train, y_train, i, edge, 2, H, G_params, d2v, n2v, gae, distance, training = True)
    for i,edge in tqdm(enumerate(node_pairs)):
        X_test = fill_matrix(X_test, None, i, edge, 1, G, G_params, d2v, n2v, gae, distance, training = False)
               
    print('Size of the matrix:', X_train.shape)
    
    return X_train, X_test, y_train

def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

def fill_matrix(X, y_train, i, edge, p, H, G_params, d2v, n2v, gae, distance, training):
    # an edge
    X[p*i,0] = len(abstracts[edge[0]]) + len(abstracts[edge[1]])
    X[p*i,1] = abs(len(abstracts[edge[0]]) - len(abstracts[edge[1]]))
    X[p*i,2] = len(set(abstracts[edge[0]]).intersection(set(abstracts[edge[1]])))
    X[p*i,3] = H.degree(edge[0]) + H.degree(edge[1])
    X[p*i,4] = abs(H.degree(edge[0]) - H.degree(edge[1]))
    X[p*i,5] = len(set(authors[edge[0]]).intersection(set(authors[edge[1]])))
    X[p*i,6] = distance(d2v[edge[0]], d2v[edge[1]])
    X[p*i,7] = distance(n2v[edge[0]], n2v[edge[1]])
    # X[p*i,8] = distance(gae.iloc[edge[0]].to_numpy(), gae.iloc[edge[1]].to_numpy())
    for idx, j in enumerate(range(8, 8 + len(G_params) + 1, 2)):
        param = G_params[idx]
        X[p*i,j] = param[edge[0]] + param[edge[1]]
        X[p*i,j+1] = abs(param[edge[0]] - param[edge[1]])
    
    if training:
        y_train[2*i] = 1

        # a randomly generated pair of nodes
        # random.seed(2*i) 
        n1 = randint(0, n-1)
        # random.seed(2*i+1) 
        n2 = randint(0, n-1)
            
        X[2*i+1,0] = len(abstracts[n1]) + len(abstracts[n2])
        X[2*i+1,1] = abs(len(abstracts[n1]) - len(abstracts[n2]))
        X[2*i+1,2] = len(set(abstracts[n1]).intersection(set(abstracts[n2])))
        X[2*i+1,3] = H.degree(n1) + H.degree(n2)
        X[2*i+1,4] = abs(H.degree(n1) - H.degree(n2))
        X[2*i+1,5] = len(set(authors[n1]).intersection(set(abstracts[n2])))
        X[2*i+1,6] = distance(d2v[n1], d2v[n2])
        X[2*i+1,7] = distance(n2v[n1], n2v[n2])
        # X[2*i+1,8] = distance(gae.iloc[n1].to_numpy(), gae.iloc[n2].to_numpy())
        for idx, j in enumerate(range(8, 8 + len(G_params) + 1, 2)):
            param = G_params[idx]
            X[2*i+1,j] = param[n1] + param[n2]
            X[2*i+1,j+1] = abs(param[n1] - param[n2])

        y_train[2*i+1] = 0

        return X, y_train
    else:
        return X


## Create Dataset

In [7]:
H = remove_random_edges_from_G(p = 0.01)

100%|██████████| 10919/10919 [00:00<00:00, 264744.41it/s]


In [8]:
# Computes node parameters for graph G and stores them in a list
G_params = graph_properties(H)

Computing graph properties


In [11]:
X_train, X_test, y_train = create_dataset(G, H, G_params = G_params)

1091955it [02:44, 6628.14it/s]
106692it [00:08, 13308.97it/s]

Size of the matrix: (2183910, 12)


Visualize dataset

In [12]:
df_train = pd.DataFrame(data=X_train)
df_test = pd.DataFrame(data=X_test)
# df_train = df_train.iloc[:,:]
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,157.0,15.0,2.0,22.0,18.0,1.0,6.734795,7.073539,50.150000,8.850000,0.000015,0.000011
1,207.0,5.0,8.0,72.0,0.0,0.0,13.894699,12.509662,100.555556,33.722222,0.000033,0.000006
2,193.0,21.0,7.0,41.0,37.0,0.0,6.000406,6.734310,38.807692,20.192308,0.000034,0.000030
3,191.0,63.0,3.0,43.0,33.0,0.0,18.291679,8.461211,164.063158,30.463158,0.000012,0.000005
4,182.0,40.0,10.0,31.0,9.0,0.0,5.918881,6.587539,32.286364,9.013636,0.000023,0.000003


Test results with cross-validation

In [13]:
clf = LogisticRegression(max_iter = 200)
scores = validation_score(clf, df_train, y_train, cv = 4, n_jobs = 7, verbose = 1)

Cross validation


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
/home/xavier/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/xavier/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:


Score: 0.23 ± 0.02


[Parallel(n_jobs=7)]: Done   4 out of   4 | elapsed:  1.4min finished


Create submission

In [14]:
clf = LogisticRegression(max_iter = 200)
create_submission(clf, df_test, df_train, y_train)

Creating submission


/home/xavier/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Submision created


## Testing embedding properties and other ideas

In [ ]:
def nodes_connected(G, u, v):
    return u in G.neighbors(v)

nodes_connected(G, 17, 0)

In [24]:
d2v = Doc2Vec.load("data/abstracts_embedding_doc2vec_vs64_w5_mc2_e100")

neighbor = nx.single_source_shortest_path_length(G, 1, cutoff=3)
# neighbor = {v: k for k, v in neighbor.items()}

for i in list(neighbor.keys())[1:10]:
    if nodes_connected(G, i, 1):
        print('Connected {:d}-{:d}: {:.2f}, {:d}'.format(i, 0, spatial.distance.cosine(d2v[i], d2v[1]), len(authors[i].intersection(authors[0]))))
    else:
        print('Not Connected {:d}-{:d}: {:.2f}, {:d}'.format(i, 0, spatial.distance.cosine(d2v[i], d2v[1]), len(authors[i].intersection(authors[0]))))

Connected 0-0: 0.09, 4
Connected 3-0: 0.59, 0
Connected 5-0: 1.10, 0
Connected 6-0: 0.65, 0
Connected 7-0: 0.61, 0
Connected 9-0: 0.69, 0
Connected 10-0: 0.70, 1
Connected 11-0: 0.54, 0
Connected 12-0: 0.61, 0


In [25]:
for i in range(0,20):
    # random.seed(i)
    k1 = randint(0, n-1)
    k2 = randint(0, n-1)
    if nodes_connected(G, k1, k2):
        print('Connected {:d}-{:d}: {:.2f}'.format(k1, 0, spatial.distance.cosine(d2v[k1], d2v[k2])))
    else:
        print('Not Connected {:d}-{:d}: {:.2f}'.format(k2, 0, spatial.distance.cosine(d2v[k1], d2v[k2])))

Not Connected 57843-0: 1.05
Not Connected 37538-0: 0.93
Not Connected 65147-0: 0.87
Not Connected 106678-0: 0.93
Not Connected 18223-0: 0.95
Not Connected 89973-0: 0.91
Not Connected 125515-0: 0.95
Not Connected 73222-0: 1.09
Not Connected 64208-0: 0.99
Not Connected 4594-0: 0.94
Not Connected 19913-0: 0.95
Not Connected 57703-0: 0.89
Not Connected 23905-0: 0.99
Not Connected 122321-0: 0.92
Not Connected 9201-0: 1.02
Not Connected 101385-0: 0.98
Not Connected 61178-0: 0.75
Not Connected 88102-0: 0.87
Not Connected 111660-0: 0.78
Not Connected 90555-0: 1.03


In [26]:
for i, edge in enumerate(G.edges()):
    print('Connected {:d}-{:d}: {:.2f}'.format(edge[0], edge[1], spatial.distance.cosine(d2v[edge[0]], d2v[edge[1]])))
    if i==30:
        break

Connected 0-1: 0.09
Connected 0-2: 0.86
Connected 1-3: 0.59
Connected 1-5: 1.10
Connected 1-6: 0.65
Connected 1-7: 0.61
Connected 1-9: 0.69
Connected 1-10: 0.70
Connected 1-11: 0.54
Connected 1-12: 0.61
Connected 1-13: 0.58
Connected 1-14: 0.59
Connected 1-15: 0.67
Connected 1-16: 0.66
Connected 1-17: 0.84
Connected 1-19: 1.00
Connected 1-20: 0.66
Connected 1-21: 0.74
Connected 1-22: 0.60
Connected 1-23: 0.61
Connected 1-24: 0.72
Connected 2-25: 0.82
Connected 2-26: 0.91
Connected 2-27: 0.99
Connected 2-28: 1.00
Connected 2-29: 0.99
Connected 2-30: 1.05
Connected 2-31: 1.04
Connected 2-32: 0.87
Connected 2-33: 0.92
Connected 2-34: 0.93


In [4]:
for i,edge in tqdm(enumerate(G.edges())):
    print(len(set(authors[edge[0]]).intersection(set(authors[edge[1]]))))
    # cos_distance = spatial.distance.cosine(d2v[edge[0]], d2v[edge[1]])
    # print(cos_distance)
   
    if i == 20:
        break

20it [00:00, 4169.29it/s]

1
0
0
0
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
